# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import xgboost as xgb

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.auxiliares_ml import downcast_dataframe, testar_modelos_com_undersampling

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

In [4]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

In [5]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### OBSERVAÇÃO:
**Podemos observar que, já que queremos detectar fraudes, ou seja o que é 1, precisamos melhorar a precisão e o recall.**

**PRÉ-PROCESSAMENTO**: Fazendo o balanceamento com OverSampler

### Utilizando o RandomOverSampler

In [6]:
# Definindo o RandomOverSampler

ros = RandomOverSampler(random_state=42, shrinkage=0.5)

# Refazendo nossa amostra

X_res_ROS, y_res_ROS = ros.fit_resample(X_train, y_train)

### XGBoost 

In [7]:
# Criando o modelo XGBoost

modelo_xgb_ROS = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb_ROS.fit(X_res_ROS, y_res_ROS)
y_pred_xgb_ROS = modelo_xgb_ROS.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb_ROS)
prec_xgb = precision_score(y_test, y_pred_xgb_ROS)
rec_xgb = recall_score(y_test, y_pred_xgb_ROS)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb_ROS)

array([[93687,   138],
       [   28,   134]])

In [8]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb_ROS))
print('Precisão:', precision_score(y_test, y_pred_xgb_ROS))
print('Recall:', recall_score(y_test, y_pred_xgb_ROS, pos_label=1))

Acurácia: 0.998233798291253
Precisão: 0.49264705882352944
Recall: 0.8271604938271605


### Random Forest

In [9]:
# Utilizando o Random Forest

clf_RF_ROS = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_res_ROS, y_res_ROS)

y_pred_RF_ROS = clf_RF_ROS.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF_ROS)

array([[93392,   433],
       [   24,   138]])

In [10]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF_ROS))
print('Precisão:', precision_score(y_test, y_pred_RF_ROS))
print('Recall:', recall_score(y_test, y_pred_RF_ROS))

Acurácia: 0.9951376254162809
Precisão: 0.24168126094570927
Recall: 0.8518518518518519


### Regressão Logística

In [11]:
# Utilizando a Regressão Logística

clfLog_ROS = LogisticRegression(random_state=0).fit(X_res_ROS, y_res_ROS)

# Calculando as métricas

y_predLog_ROS = clfLog_ROS.predict(X_test)
y_pred_probaLog_ROS = clfLog_ROS.predict_proba(X_test)[:,1]

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_predLog_ROS)

array([[90553,  3272],
       [   15,   147]])

In [12]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predLog_ROS))
print('Precisão:', precision_score(y_test, y_predLog_ROS))
print('Recall:', recall_score(y_test, y_predLog_ROS))

Acurácia: 0.9650270782129443
Precisão: 0.042995027785902314
Recall: 0.9074074074074074


### Utilizando o SMOTE

In [13]:
# Definindo o SMOTE

sm = SMOTE(random_state=42)

# Refazendo nossa amostra

X_res_SM, y_res_SM = sm.fit_resample(X_train, y_train)

### XGBoost 

In [14]:
# Criando o modelo XGBoost

modelo_xgb_SM = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb_SM.fit(X_res_SM, y_res_SM)
y_pred_xgb_SM = modelo_xgb_SM.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb_SM)
prec_xgb = precision_score(y_test, y_pred_xgb_SM)
rec_xgb = recall_score(y_test, y_pred_xgb_SM)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb_SM)

array([[93777,    48],
       [   29,   133]])

In [15]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb_SM))
print('Precisão:', precision_score(y_test, y_pred_xgb_SM))
print('Recall:', recall_score(y_test, y_pred_xgb_SM, pos_label=1))

Acurácia: 0.9991807377616053
Precisão: 0.7348066298342542
Recall: 0.8209876543209876


### Random Forest

In [16]:
# Utilizando o Random Forest

clf_RF_SM = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_res_SM, y_res_SM)

y_pred_RF_SM = clf_RF_SM.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF_SM)

array([[93601,   224],
       [   25,   137]])

In [17]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF_SM))
print('Precisão:', precision_score(y_test, y_pred_RF_SM))
print('Recall:', recall_score(y_test, y_pred_RF_SM))

Acurácia: 0.9973506974368795
Precisão: 0.37950138504155123
Recall: 0.845679012345679


### Regressão Logística

In [18]:
# Utilizando a Regressão Logística

clfLog_SM = LogisticRegression(random_state=0).fit(X_res_SM, y_res_SM)

# Calculando as métricas

y_predLog_SM = clfLog_SM.predict(X_test)
y_pred_probaLog_SM = clfLog_SM.predict_proba(X_test)[:,1]

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_predLog_SM)

array([[91989,  1836],
       [   18,   144]])

In [19]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predLog_SM))
print('Precisão:', precision_score(y_test, y_predLog_SM))
print('Recall:', recall_score(y_test, y_predLog_SM))

Acurácia: 0.9802738676625491
Precisão: 0.07272727272727272
Recall: 0.8888888888888888


### OBSERVAÇÕES: 

- Em geral os resultados foram melhores do que com UnderSampler.
- Ainda há um grande desequilíbrio entre precisão e recall.

**Os melhores resultados foram**:

- **1º Lugar: XGBoost (Utilizando o SMOTE)**

    Acurácia: 0.9991807377616053 / Precisão: 0.7348066298342542 / Recall: 0.8209876543209876

- **2º Lugar: XGBoost (Utilizando o RandomOverSampler)**

    Acurácia: 0.998233798291253 / Precisão: 0.49264705882352944 / Recall: 0.8271604938271605

- **3º Lugar: Random Forest (Utilizando o RandomOverSampler)** <br>

    Acurácia: 0.9951376254162809 / Precisão: 0.24168126094570927 / Recall: 0.8518518518518519190576